# Data for Model Building and Validation

In [2]:
import wget

In [53]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re, string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ktrain
from ktrain import text
import cleantext
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import xgboost as xgb 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.utils.deprecation import _raise_dep_warning_if_not_pytest




In [3]:
wget.download('https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py')

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

100% [..............................................................................] 10246 / 10246

In [4]:
import wget

url ="https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
wget.download(url)

100% [............................................................................] 607343 / 607343

'nlp_getting_started (2).zip'

In [5]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys
# Unzip data
unzip_data("nlp_getting_started.zip")

In [21]:
import pandas as pd
#df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names = ['target','ids','date','flag','user','text'])

train_df = pd.read_excel('trainp.xlsx')
train_df.head()

,textID,text,selected_text,target
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,1
2,088c60f138,my boss is bullying me...,bullying me,1
3,9642c003ef,what interview! leave me alone,leave me alone,1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",1


In [22]:
#train_df.target = train_df.target.astype(int)
#train_df["target"] = pd.to_numeric(train_df["target"], downcast="float", errors='coerce')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27446 non-null  object
 3   target         27481 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 858.9+ KB


In [8]:
train_df.target.value_counts()

0.0    19700
1.0     7781
Name: target, dtype: int64

In [25]:
train_df.text=train_df.text.astype(str)

In [26]:
import re, string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # miniscule
    text = text.lower()
    
    #remove emoji
    text = text.encode("ascii", "ignore").decode()
    
    # removing usernames
    text = re.sub('@[^\s]+', '', text)

    # removing links
    text = re.sub('((https?://[^\s]+)|(www\.[^\s]+))','',text)
    
        # removing rt
    text = re.sub('rt','',text)
    
    # removing punctuation
    text = text.translate(str.maketrans('','',string.punctuation))
    
    # tokenization
    tokens = word_tokenize(text)
   
    # suppression of stop words and lemmatization
    wordLemm = WordNetLemmatizer()
    final_words = []
    for token in tokens:
      # suppression of stop words
      if token not in stop_words and len(token) > 1:
        lemm = wordLemm.lemmatize(token)
        final_words.append(lemm)
    return ' '.join(final_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jelee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jelee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jelee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
train_df['cleantext'] = train_df.text.apply(clean_text)

In [15]:
train_df.head()

,textID,text,selected_text,target,cleantext
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",0.0,id responded going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,1.0,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,1.0,bos bullying
3,9642c003ef,what interview! leave me alone,leave me alone,1.0,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",1.0,son couldnt put release already bought


In [30]:
#Data Clean
import pandas as pd
import ktrain
from ktrain import text
import cleantext


In [31]:
# clean training text
l=len(train_df)
display(l)
cleanlist=[]
textlength=[]
for i in range(l):
    ct=cleantext.clean(train_df.iloc[i,3], clean_all= True)
    cleanlist.append(ct)
    lct=len(ct)
    textlength.append(lct)

7613

In [33]:
df_clean=pd.DataFrame(cleanlist)
df_clean.columns=['cleantext']
frames=[train_df,df_clean]
newdf=pd.concat(frames, axis=1)
display(newdf)

,id,keyword,location,text,target,cleantext
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridg collaps nearbi home...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,ariaahrari thetawniest control wild fire calif...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utckm volcano hawaii httptcozdtoydebj
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,polic investig ebik collid car littl portug eb...


In [28]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,textID,text,selected_text,target,cleantext
1588,a7f72a928a,WOOOOOOOOOO are you coming to Nottingham at...,t? lovelovelove,0,woooooooooo coming nottingham point lovelovelove3
23879,ef42dee96c,resting had a whole day of walking,resting had a whole day of walking,0,resting whole day walking
6561,07d17131b1,"was in Palawan a couple of days ago, i`ll try ...","was in Palawan a couple of days ago, i`ll try ...",0,palawan couple day ago ill try post picture tom
2602,2820205db5,I know! I`m so slow its horrible. DON`T TELL ...,horrible.,1,know im slow horrible dont tell
4003,7d3ce4363c,"Glad I went out, glad I didn`t leave early, an...",glad,0,glad went glad didnt leave early glad afterpay...


In [29]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["cleantext"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [30]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(24732, 24732, 2749, 2749)

In [55]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [56]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [57]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

# Model0: MultinomialNB

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [32]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [33]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [34]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 76.31866133139323,
 'precision': 0.7923656577921231,
 'recall': 0.7631866133139323,
 'f1': 0.6942348800890389}

# Model_0_1:XGBoost

In [35]:
import xgboost as xgb 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0_1 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", xgb.XGBClassifier()) # model the text
])

# Fit the pipeline to the training data
model_0_1.fit(train_sentences, train_labels)

D:\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:39:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=4, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [36]:
# Make predictions
baseline01_preds = model_0_1.predict(val_sentences)
baseline01_preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [37]:
# Get baseline results
baseline01_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline01_preds)
baseline01_results

{'accuracy': 82.06620589305203,
 'precision': 0.8168084487553012,
 'recall': 0.8206620589305202,
 'f1': 0.8033551638746379}

# RidgeClassifier

In [38]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [47]:
# Create tokenization and modelling pipeline
model_0_2 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", linear_model.RidgeClassifier(class_weight='balanced')) # model the text
])

# Fit the pipeline to the training data
model_0_2.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RidgeClassifier(class_weight='balanced'))])

In [48]:
# Make predictions
baseline02_preds = model_0_2.predict(val_sentences)
baseline02_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline02_preds)
baseline02_results

{'accuracy': 69.84357948344852,
 'precision': 0.780520237628925,
 'recall': 0.6984357948344853,
 'f1': 0.7149417345145571}

# MLPClassifier

In [49]:
from sklearn.neural_network import MLPClassifier

In [77]:
# Create tokenization and modelling pipeline
model_0_3 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MLPClassifier(max_iter=300, random_state=42)) # model the text
])

# Fit the pipeline to the training data
model_0_3.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', MLPClassifier(max_iter=300, random_state=42))])

In [79]:
# Make predictions
baseline03_preds = model_0_3.predict(val_sentences)
baseline03_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline03_preds)
baseline03_results

{'accuracy': 79.01054929065114,
 'precision': 0.7912733684492507,
 'recall': 0.7901054929065114,
 'f1': 0.7906703375255775}

# USE MODEL

In [71]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 


INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 210.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 420.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 620.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 840.00MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


In [72]:
from tensorflow.keras import layers
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [73]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels)
                              )

Epoch 1/10
773/773 [==============================] - 131s 152ms/step - loss: 0.4462 - accuracy: 0.7927 - val_loss: 0.4061 - val_accuracy: 0.8192
Epoch 2/10
773/773 [==============================] - 5s 7ms/step - loss: 0.4056 - accuracy: 0.8147 - val_loss: 0.4004 - val_accuracy: 0.8210
Epoch 3/10
773/773 [==============================] - 6s 8ms/step - loss: 0.3961 - accuracy: 0.8195 - val_loss: 0.3998 - val_accuracy: 0.8196
Epoch 4/10
773/773 [==============================] - 5s 7ms/step - loss: 0.3865 - accuracy: 0.8269 - val_loss: 0.3975 - val_accuracy: 0.8247
Epoch 5/10
773/773 [==============================] - 5s 7ms/step - loss: 0.3767 - accuracy: 0.8316 - val_loss: 0.3958 - val_accuracy: 0.8236
Epoch 6/10
773/773 [==============================] - 6s 8ms/step - loss: 0.3667 - accuracy: 0.8388 - val_loss: 0.3961 - val_accuracy: 0.8250
Epoch 7/10
773/773 [==============================] - 5s 7ms/step - loss: 0.3577 - accuracy: 0.8437 - val_loss: 0.3965 - val_accuracy: 0.8250
Ep

In [74]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)

In [75]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.], dtype=float32)>

In [76]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 82.57548199345216,
 'precision': 0.8196212938957227,
 'recall': 0.8257548199345216,
 'f1': 0.8210058029142369}

# Feed-forward neural network (dense model)

In [51]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [58]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [59]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220302-121615
Epoch 1/5
773/773 [==============================] - 18s 20ms/step - loss: 0.5158 - accuracy: 0.7489 - val_loss: 0.4294 - val_accuracy: 0.8025
Epoch 2/5
773/773 [==============================] - 14s 18ms/step - loss: 0.3751 - accuracy: 0.8402 - val_loss: 0.4039 - val_accuracy: 0.8236
Epoch 3/5
773/773 [==============================] - 17s 22ms/step - loss: 0.3217 - accuracy: 0.8688 - val_loss: 0.4135 - val_accuracy: 0.8210
Epoch 4/5
773/773 [==============================] - 14s 18ms/step - loss: 0.2905 - accuracy: 0.8854 - val_loss: 0.4322 - val_accuracy: 0.8192
Epoch 5/5
773/773 [==============================] - 13s 17ms/step - loss: 0.2676 - accuracy: 0.8962 - val_loss: 0.4492 - val_accuracy: 0.8178


In [61]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 81.77519097853765,
 'precision': 0.8103476894663983,
 'recall': 0.8177519097853765,
 'f1': 0.8116572297344858}

# LSTM

In [98]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [99]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [100]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20220303-101847
Epoch 1/5
773/773 [==============================] - 130s 137ms/step - loss: 0.4513 - accuracy: 0.7978 - val_loss: 0.4348 - val_accuracy: 0.8138
Epoch 2/5
773/773 [==============================] - 21s 27ms/step - loss: 0.3283 - accuracy: 0.8648 - val_loss: 0.4031 - val_accuracy: 0.8192
Epoch 3/5
773/773 [==============================] - 21s 27ms/step - loss: 0.2668 - accuracy: 0.8939 - val_loss: 0.4959 - val_accuracy: 0.8163
Epoch 4/5
773/773 [==============================] - 23s 30ms/step - loss: 0.2130 - accuracy: 0.9150 - val_loss: 0.5097 - val_accuracy: 0.8090
Epoch 5/5
773/773 [==============================] - 22s 28ms/step - loss: 0.1706 - accuracy: 0.9310 - val_loss: 0.6188 - val_accuracy: 0.8007


In [101]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)

model_2_results 

{'accuracy': 80.06547835576573,
 'precision': 0.7969378617204406,
 'recall': 0.8006547835576573,
 'f1': 0.7985217060804096}

# GRU

In [102]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [103]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [104]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20220303-102406
Epoch 1/5
773/773 [==============================] - 27s 28ms/step - loss: 0.4641 - accuracy: 0.7891 - val_loss: 0.4145 - val_accuracy: 0.8236
Epoch 2/5
773/773 [==============================] - 19s 24ms/step - loss: 0.3276 - accuracy: 0.8658 - val_loss: 0.3960 - val_accuracy: 0.8247
Epoch 3/5
773/773 [==============================] - 18s 23ms/step - loss: 0.2604 - accuracy: 0.8976 - val_loss: 0.4716 - val_accuracy: 0.8203
Epoch 4/5
773/773 [==============================] - 18s 23ms/step - loss: 0.1942 - accuracy: 0.9228 - val_loss: 0.5369 - val_accuracy: 0.8094
Epoch 5/5
773/773 [==============================] - 16s 21ms/step - loss: 0.1484 - accuracy: 0.9398 - val_loss: 0.6473 - val_accuracy: 0.8043


In [105]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 80.4292469989087,
 'precision': 0.7980248973105916,
 'recall': 0.804292469989087,
 'f1': 0.8001609517020097}

# Predict data with 3 models built
Naive Bayes
USE
Dense Model

In [108]:
df=pd.read_excel('f:/cvds.xlsx')
df.head()

,id,text,created_at,lang,location,user_id,event,usa,usar,nevent,...,joy,sadness,surprise,trust,negative,positive,sentiment,vader,BERT,text_len
0,0,RT @EbonyT_PhD:Shout out to the nurses who???...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,718000000000000000.0,omicron,NaN,NaN,2.0,...,0,0,0,1,1,3,0.113636,0.8523,0.0,176
1,1,RT @GOPKeegan: True! #Democrats #COVID19 #Trut...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,1450000000000000000.0,omicron,NaN,NaN,2.0,...,1,0,0,2,0,2,0.350000,0.6588,0.0,31
2,2,"RT @nilikm:In other words, daily #COVID19 rep...",Wed Dec 29 18:01:40 +0000 2021,en,"Mississauga, Ontario",1280000000000000000.0,omicron,NaN,0.0,2.0,...,0,0,0,3,2,4,0.000000,-0.6440,0.0,158
3,3,It?????s Wednesday. _x000D_\n_x000D_\nScho...,Wed Dec 29 18:01:41 +0000 2021,en,at least 2m away from you,26824287,omicron,NaN,0.0,2.0,...,0,0,0,1,0,0,-0.600000,0.0000,0.0,76
4,4,RT @VikiLovesFACS:How can we know that #COVID1...,Wed Dec 29 18:01:42 +0000 2021,en,"Chicago, IL",1330000000000000000.0,omicron,il,1.0,2.0,...,0,0,0,0,1,0,0.050000,0.5873,0.0,79


In [111]:
import numpy as np

ct=np.asarray(df['cleantext'].astype(str))

model_1_pred_probs = model_1.predict(ct)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions

model_0_1preds = model_0_1.predict(ct)

# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(ct)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs)) # squeeze removes single dimensions


In [156]:
len(model_0_preds)

103620

In [113]:
df1=pd.DataFrame(model_0_1preds)
df1.columns=['XG_p']
df2=pd.DataFrame(model_1_preds)
df2.columns=['DENSE_p']
df3=pd.DataFrame(model_6_preds)
df3.columns=['USE_p']

frames=[df,df1, df2, df3 ]
df=pd.concat(frames, axis=1)
display(df.head())andas 

,id,text,created_at,lang,location,user_id,event,usa,usar,nevent,...,trust,negative,positive,sentiment,vader,BERT,text_len,XG_p,DENSE_p,USE_p
0,0,RT @EbonyT_PhD:Shout out to the nurses who???...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,718000000000000000.0,omicron,NaN,NaN,2.0,...,1,1,3,0.113636,0.8523,0.0,176,1,0.0,0.0
1,1,RT @GOPKeegan: True! #Democrats #COVID19 #Trut...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,1450000000000000000.0,omicron,NaN,NaN,2.0,...,2,0,2,0.350000,0.6588,0.0,31,0,0.0,0.0
2,2,"RT @nilikm:In other words, daily #COVID19 rep...",Wed Dec 29 18:01:40 +0000 2021,en,"Mississauga, Ontario",1280000000000000000.0,omicron,NaN,0.0,2.0,...,3,2,4,0.000000,-0.6440,0.0,158,1,1.0,0.0
3,3,It?????s Wednesday. _x000D_\n_x000D_\nScho...,Wed Dec 29 18:01:41 +0000 2021,en,at least 2m away from you,26824287,omicron,NaN,0.0,2.0,...,1,0,0,-0.600000,0.0000,0.0,76,0,1.0,0.0
4,4,RT @VikiLovesFACS:How can we know that #COVID1...,Wed Dec 29 18:01:42 +0000 2021,en,"Chicago, IL",1330000000000000000.0,omicron,il,1.0,2.0,...,0,1,0,0.050000,0.5873,0.0,79,0,0.0,0.0


In [62]:
df.to_csv('f:\df_sent.csv')

# Rule_based Sentiment

In [146]:
import pandas as pd
df=pd.read_csv('df_sent.csv', encoding='utf8')
df.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1.1,text,created_at,lang,location,user_id,event,usa,usar,nevent,cleantext,MNB,DENSE,USE,BERT,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
0,0,0,RT @EbonyT_PhD:Shout out to the nurses who???...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,718000000000000000.0,omicron,NaN,False,2.0,nurse who?ve told can?t drink water nurse st...,0.0,0.0,0.0,not_target,0,1,0,0,0,0,0,1,1,3
1,1,1,RT @GOPKeegan: True! #Democrats #COVID19 #Trut...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,1450000000000000000.0,omicron,NaN,False,2.0,true democrat covid19 truth covid,0.0,0.0,0.0,not_target,0,0,0,0,1,0,0,2,0,2
2,2,2,"RT @nilikm:In other words, daily #COVID19 rep...",Wed Dec 29 18:01:40 +0000 2021,en,"Mississauga, Ontario",1280000000000000000.0,omicron,NaN,False,2.0,word daily covid19 repos gross underestimate t...,0.0,0.0,0.0,not_target,0,1,2,1,0,0,0,3,2,4
3,3,3,It???s Wednesday. \r\n\r\nSchool starts on Mo...,Wed Dec 29 18:01:41 +0000 2021,en,at least 2m away from you,26824287.0,omicron,NaN,False,2.0,it?s wednesday school stas monday one 430pm f...,0.0,0.0,0.0,not_target,0,0,0,0,0,0,0,1,0,0
4,4,4,RT @VikiLovesFACS:How can we know that #COVID1...,Wed Dec 29 18:01:42 +0000 2021,en,"Chicago, IL",1330000000000000000.0,omicron,il,True,2.0,know covid19 vaccine harm feility could effect...,0.0,0.0,0.0,not_target,0,0,0,1,0,0,0,0,1,0


In [45]:
from textblob import TextBlob

def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

df['sentiment'] = df['cleantext'].apply(sentiment_calc)

In [46]:
df

,Unnamed: 0,Unnamed: 0.1.1,text,created_at,lang,location,user_id,event,usa,usar,nevent,cleantext,MNB,DENSE,USE,BERT,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive,sentiment
0,0,0,RT @EbonyT_PhD:Shout out to the nurses who???...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,718000000000000000.0,omicron,NaN,False,2.0,nurse who?ve told can?t drink water nurse st...,0.0,0.0,0.0,not_target,0,1,0,0,0,0,0,1,1,3,0.113636
1,1,1,RT @GOPKeegan: True! #Democrats #COVID19 #Trut...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,1450000000000000000.0,omicron,NaN,False,2.0,true democrat covid19 truth covid,0.0,0.0,0.0,not_target,0,0,0,0,1,0,0,2,0,2,0.350000
2,2,2,"RT @nilikm:In other words, daily #COVID19 rep...",Wed Dec 29 18:01:40 +0000 2021,en,"Mississauga, Ontario",1280000000000000000.0,omicron,NaN,False,2.0,word daily covid19 repos gross underestimate t...,0.0,0.0,0.0,not_target,0,1,2,1,0,0,0,3,2,4,0.000000
3,3,3,It???s Wednesday. \r\n\r\nSchool starts on Mo...,Wed Dec 29 18:01:41 +0000 2021,en,at least 2m away from you,26824287.0,omicron,NaN,False,2.0,it?s wednesday school stas monday one 430pm f...,0.0,0.0,0.0,not_target,0,0,0,0,0,0,0,1,0,0,-0.600000
4,4,4,RT @VikiLovesFACS:How can we know that #COVID1...,Wed Dec 29 18:01:42 +0000 2021,en,"Chicago, IL",1330000000000000000.0,omicron,il,True,2.0,know covid19 vaccine harm feility could effect...,0.0,0.0,0.0,not_target,0,0,0,1,0,0,0,0,1,0,0.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103619,103619,103615,RT @HeadUNDRR: There may be more to come in wh...,Mon Aug 31 16:07:27 +0000 2020,en,"Newcastle Upon Tyne, England",285463199.0,laura,NaN,False,NaN,may come already active atlantic hurricane sea...,0.0,1.0,1.0,target,0,0,0,0,0,0,0,0,0,0,-0.133333
103620,103620,103616,No one should wish for 4 more years of reckles...,Mon Aug 31 16:07:28 +0000 2020,en,NaN,880965564.0,laura,NaN,False,NaN,one wish year reckless endangerment,0.0,0.0,0.0,not_target,1,1,0,2,0,0,0,0,2,0,0.000000
103621,103621,103617,It's #Tornado &amp; #HurricaneSeason ...Double...,Mon Aug 31 16:07:28 +0000 2020,en,South Florida,1681212607.0,laura,florida,True,NaN,tornado amp hurricaneseason double whammy get ...,1.0,1.0,1.0,target,0,0,0,1,0,0,0,0,0,0,0.100000
103622,103622,103618,"RT @JoeBiden: There?s a lot happening, but I h...",Mon Aug 31 16:07:30 +0000 2020,en,"Madison, WI",464204423.0,laura,wi,True,NaN,there lot happening hope dont overlook brave f...,0.0,1.0,1.0,target,0,2,0,1,1,0,1,1,0,1,0.800000


In [48]:
pip install vaderSentiment

In [70]:


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_calc(text):
    try:
        return analyzer.polarity_scores(text)['compound']
    
    except:
        return None

df['vader'] = df['text'].apply(vader_calc)

df

,Unnamed: 0,Unnamed: 0.1.1,text,created_at,lang,location,user_id,event,usa,usar,nevent,cleantext,MNB,DENSE,USE,BERT,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive,sentiment,vader
0,0,0,RT @EbonyT_PhD:Shout out to the nurses who???...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,718000000000000000.0,omicron,NaN,False,2.0,nurse who?ve told can?t drink water nurse st...,0.0,0.0,0.0,not_target,0,1,0,0,0,0,0,1,1,3,0.113636,0.8523
1,1,1,RT @GOPKeegan: True! #Democrats #COVID19 #Trut...,Wed Dec 29 18:01:40 +0000 2021,en,NaN,1450000000000000000.0,omicron,NaN,False,2.0,true democrat covid19 truth covid,0.0,0.0,0.0,not_target,0,0,0,0,1,0,0,2,0,2,0.350000,0.6588
2,2,2,"RT @nilikm:In other words, daily #COVID19 rep...",Wed Dec 29 18:01:40 +0000 2021,en,"Mississauga, Ontario",1280000000000000000.0,omicron,NaN,False,2.0,word daily covid19 repos gross underestimate t...,0.0,0.0,0.0,not_target,0,1,2,1,0,0,0,3,2,4,0.000000,-0.6440
3,3,3,It???s Wednesday. \r\n\r\nSchool starts on Mo...,Wed Dec 29 18:01:41 +0000 2021,en,at least 2m away from you,26824287.0,omicron,NaN,False,2.0,it?s wednesday school stas monday one 430pm f...,0.0,0.0,0.0,not_target,0,0,0,0,0,0,0,1,0,0,-0.600000,0.0000
4,4,4,RT @VikiLovesFACS:How can we know that #COVID1...,Wed Dec 29 18:01:42 +0000 2021,en,"Chicago, IL",1330000000000000000.0,omicron,il,True,2.0,know covid19 vaccine harm feility could effect...,0.0,0.0,0.0,not_target,0,0,0,1,0,0,0,0,1,0,0.050000,0.5873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103619,103619,103615,RT @HeadUNDRR: There may be more to come in wh...,Mon Aug 31 16:07:27 +0000 2020,en,"Newcastle Upon Tyne, England",285463199.0,laura,NaN,False,NaN,may come already active atlantic hurricane sea...,0.0,1.0,1.0,target,0,0,0,0,0,0,0,0,0,0,-0.133333,0.4576
103620,103620,103616,No one should wish for 4 more years of reckles...,Mon Aug 31 16:07:28 +0000 2020,en,NaN,880965564.0,laura,NaN,False,NaN,one wish year reckless endangerment,0.0,0.0,0.0,not_target,1,1,0,2,0,0,0,0,2,0,0.000000,-0.3535
103621,103621,103617,It's #Tornado &amp; #HurricaneSeason ...Double...,Mon Aug 31 16:07:28 +0000 2020,en,South Florida,1681212607.0,laura,florida,True,NaN,tornado amp hurricaneseason double whammy get ...,1.0,1.0,1.0,target,0,0,0,1,0,0,0,0,0,0,0.100000,0.4995
103622,103622,103618,"RT @JoeBiden: There?s a lot happening, but I h...",Mon Aug 31 16:07:30 +0000 2020,en,"Madison, WI",464204423.0,laura,wi,True,NaN,there lot happening hope dont overlook brave f...,0.0,1.0,1.0,target,0,2,0,1,1,0,1,1,0,1,0.800000,0.4922
